## Makespan Theorem (1)
An objective to minimize makespan can be considered to induce a complete order upon two separation identical aircraft $i$ and $j$ if $r_i \leq r_j$. That is, the makespan of any partial sequence $s$, which schedules aircraft $i$ before aircraft $j$ will result in a makespan no worse than in $s\prime$ (schedulling $j$ after $i$); therefore aircraft $i$ should always be scheduled before aircraft $j$. Firstly, import Z3 Theorem Prover and the itertools library and instantiate a Solver instance.


In [85]:
from z3 import *

s = Solver()

Then, we define the following:
- $\delta \in \mathbb{Z} \to \mathbb{Z} \to \mathbb{R}$
- $r_i \in \mathbb{Z} \to \mathbb{R}$
- $S = \{p_1,\ a,\ p_2,\ b,\ p_3\}$

then
1. Non-negativity of release times / $r_x$:
$$ r_x \geq 0 \quad \forall x \in S $$

2. Non-negativity of $\delta$
$$ \delta(x, y) \geq 0 \quad \forall x \in (S \times S) $$

3. Identical separations between `a` and `b` $\delta_a = \delta_b$
$$ \delta(a, x) = \delta(b, x) \wedge \delta(x, a) = \delta(x, b) \quad \forall x \in S $$


In [90]:
# Functions
δ = Function('δ', IntSort(), IntSort(), RealSort())
R = Function('R', IntSort(), RealSort())
B = Function('B', IntSort(), RealSort())

# Aircraft symbols
p1, a, p2, b, p3 = Ints('p1 a p2 b p3')
ac = [p1, a, p2, b, p3]


# Basic constraints
s.add(ForAll([a], R(a) >= 0))
s.add(ForAll([a], B(a) >= 0))
s.add(ForAll([a], R(a) >= B(a)))
s.add(ForAll([a, b], δ(a, b) >= 0))

# a and b are δ-identical
for x in ac:
    s.add(δ(a, x) == δ(b, x))  # outgoing
    s.add(δ(x, a) == δ(x, b))  # incoming




Then define some helper functions to encode a $max$ function symbolically. Define a recursive function to compute a symbolic takeoff time for a given sequence.

In [91]:

# Helper symbolic max functions
def zmax(x, y): return If(x >= y, x, y)
def zmax_list(xs):
    if not xs:
        raise ValueError("zmax_list requires at least one element")
    m = xs[0]
    for x in xs[1:]:
        m = If(x >= m, x, m)
    return m

# Compute symbolic takeoff time given a sequence
def compute_T(seq):
    T = {}
    T[seq[0]] = R(seq[0])
    for i in range(1, len(seq)):
        preds = seq[:i]
        T[seq[i]] = zmax(R(seq[i]),
                         zmax_list([T[x] + δ(x, seq[i]) for x in preds])
        )
    return T

Define $s$ and $s'$ as sample sequences, in this $p_1, p_2, p_3$ represent subsequences (but represented as a singular aircraft). These needs some verification.


$$ s = \{ p_1,\ a,\ p_2,\ b,\ p_3 \} $$
$$ s' = \{ p_1,\ b,\ p_2,\ a,\ p_3 \} $$

Then we define makespan of $s$ ($m$) and makespan of $s'$ ($m'$) as the takeoff time of $p_3$, or $T(p_3)$. In the paper, the pruning rule is also dependent on:

$$ r_a \leq r_b $$

so this is added as a constraint. Lastly, $m > m'$ is added as a constraint as the unsat case.


In [92]:
# Sequences
S1 = [p1, a, p2, b, p3]
S2 = [p1, b, p2, a, p3]

T1 = compute_T(S1)
T2 = compute_T(S2)

# Compare makespans (symbolically)
m = T1[S1[-1]]
m_ = T2[S2[-1]]

# Release time of plane A is less than release time of plane B
s.add(R(a) <= R(b))

# Theorem: makespan of a, b always less than, or equal to, makespan of b, a (find a counter example)
s.add(m > m_)


Check the theorem prover

In [93]:
if s.check() == z3.sat:
    model = s.model()
    print("\nTheorem holds SAT (counterexample found).\nModel:")
    for d in model.decls():
        print(f" - {d.name()} = {model[d]}")
elif s.check() == z3.unsat:
    print("\nTheorem holds: UNSAT (no counterexample found)")


Theorem holds: UNSAT (no counterexample found)
